In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🛒 Measuring Real GDP with Changing Prices & Quantities

When comparing economic output between two periods (Year 0 and Year 1), we need to account for changes in both the **quantities** produced and the **prices** at which they are sold.

* **Nominal GDP** reflects the total value using the prices prevailing *in each year*.
* **Real GDP** aims to measure the change in output using *constant prices* from a chosen **base year**.

This simulation calculates Nominal and Real GDP for a simple two-good economy (e.g., Tacos and Tablets) where both prices and quantities can change, using Year 0 as the fixed base year for calculating Real GDP.

# 🔢 Calculations (Base Year = Year 0)

Let $P_{i,t}$ and $Q_{i,t}$ be the price and quantity of good $i$ (1=Good 1, 2=Good 2) in year $t$ (0 or 1).

1.  **Nominal GDP:** Value using current prices and quantities for each year.
    * Nominal GDP$_0 = P_{1,0} Q_{1,0} + P_{2,0} Q_{2,0}$
    * Nominal GDP$_1 = P_{1,1} Q_{1,1} + P_{2,1} Q_{2,1}$

2.  **Real GDP (Base Year 0):** Value using current quantities but **Year 0 prices** for both years.
    * Real GDP$_0^{\text{Base 0}} = P_{1,0} Q_{1,0} + P_{2,0} Q_{2,0}$ *(Same as Nominal GDP$_0$)*
    * Real GDP$_1^{\text{Base 0}} = P_{1,0} Q_{1,1} + P_{2,0} Q_{2,1}$

3.  **Growth Rates:**
    * Nominal Growth Rate ($g_N$) = $(\frac{\text{Nominal GDP}_1}{\text{Nominal GDP}_0} - 1) \times 100\%$
    * Real Growth Rate ($g_R$) = $(\frac{\text{Real GDP}_1^{\text{Base 0}}}{\text{Real GDP}_0^{\text{Base 0}}} - 1) \times 100\%$

4.  **GDP Deflator:** An index measuring the price level in Year 1 relative to the base Year 0.
    * GDP Deflator$_0 = 100$ (by definition)
    * GDP Deflator$_1 = \frac{\text{Nominal GDP}_1}{\text{Real GDP}_1^{\text{Base 0}}} \times 100$

5.  **Inflation Rate ($\pi$):** The percentage change in the GDP Deflator.
    $$ \pi \approx \frac{\text{Deflator}_1 - \text{Deflator}_0}{\text{Deflator}_0} \times 100\% = (\text{Deflator}_1 / 100 - 1) \times 100\% $$

We expect the relationship $g_N \approx g_R + \pi$ to hold approximately. The simulation calculates these values based on the inputs provided.

# 🧮 Calculation: Base-Year Method

Let $P_{i,t}$ and $Q_{i,t}$ be the price and quantity of good $i$ in year $t$.

---

## 📘 Steps to Calculate Real GDP

1. **Choose a Base Year:**  
   Select one year (e.g., Year 1 in this simulation) whose prices will be used for all calculations of Real GDP.

2. **Calculate Nominal GDP:**  
   Compute the value of production in each year using that year’s *own* prices.
   - $ \text{Nominal GDP}_{Y1} = P_{\text{apples}, Y1} \cdot Q_{\text{apples}, Y1} + P_{\text{bananas}, Y1} \cdot Q_{\text{bananas}, Y1} $
   - $ \text{Nominal GDP}_{Y2} = P_{\text{apples}, Y2} \cdot Q_{\text{apples}, Y2} + P_{\text{bananas}, Y2} \cdot Q_{\text{bananas}, Y2} $

3. **Calculate Real GDP:**  
   Use the *base year’s prices* (Year 1 prices here) for all years.
   - $ \text{Real GDP}_{Y1} = P_{\text{apples}, Y1} \cdot Q_{\text{apples}, Y1} + P_{\text{bananas}, Y1} \cdot Q_{\text{bananas}, Y1} $  
     *(Note: Real GDP in the base year equals Nominal GDP in the base year)*
   - $ \text{Real GDP}_{Y2} = P_{\text{apples}, Y1} \cdot Q_{\text{apples}, Y2} + P_{\text{bananas}, Y1} \cdot Q_{\text{bananas}, Y2} $

4. **Calculate Real GDP Growth Rate:**  
   Find the percentage change between the Real GDP values:

   $$
   g_{\text{Real}} = \frac{\text{Real GDP}_{Y2} - \text{Real GDP}_{Y1}}{\text{Real GDP}_{Y1}} \times 100\%
   $$

---

## 🧮 GDP Deflator

This method also allows us to calculate an implicit price index — the **GDP Deflator** — for Year 2 (relative to the base year, where Y1 = 100):

$$
\text{GDP Deflator}_{Y2} = \frac{\text{Nominal GDP}_{Y2}}{\text{Real GDP}_{Y2}} \times 100
$$

The inflation rate between Y1 and Y2 is approximately the percentage change in the GDP deflator.

> 📌 *Note: This simulation assumes prices remain constant between Y1 and Y2 as set by the sliders, focusing only on quantity changes. A more advanced version would include changing prices to highlight inflation more clearly.*

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def compare_nom_real_gdp_two_goods(
    # Year 0 (Base Year)
    p_good1_0=2.0, q_good1_0=100,
    p_good2_0=500.0, q_good2_0=5,
    # Year 1
    p_good1_1=3.0, q_good1_1=120,
    p_good2_1=550.0, q_good2_1=6
    ):
    """
    Calculates Nominal GDP, Real GDP (Base Year 0), GDP Deflator,
    and growth rates for a two-good economy with changing prices/quantities.

    Args:
        p_good1_0, q_good1_0: Price and Quantity of Good 1 in Year 0.
        p_good2_0, q_good2_0: Price and Quantity of Good 2 in Year 0.
        p_good1_1, q_good1_1: Price and Quantity of Good 1 in Year 1.
        p_good2_1, q_good2_1: Price and Quantity of Good 2 in Year 1.
    """
    # --- Calculations ---
    # Nominal GDP
    nom_gdp_0 = (p_good1_0 * q_good1_0) + (p_good2_0 * q_good2_0)
    nom_gdp_1 = (p_good1_1 * q_good1_1) + (p_good2_1 * q_good2_1)

    # Real GDP (using Year 0 prices as base)
    real_gdp_0 = nom_gdp_0 # Real GDP = Nominal GDP in base year
    real_gdp_1 = (p_good1_0 * q_good1_1) + (p_good2_0 * q_good2_1)

    # Growth Rates (handle division by zero)
    g_nominal = np.nan
    g_real = np.nan
    if nom_gdp_0 > 1e-9:
        g_nominal = (nom_gdp_1 / nom_gdp_0 - 1) * 100
        g_real = (real_gdp_1 / real_gdp_0 - 1) * 100


    # GDP Deflator & Inflation
    deflator_0 = 100.0
    deflator_1 = np.nan
    inflation = np.nan
    if real_gdp_1 > 1e-9:
        deflator_1 = (nom_gdp_1 / real_gdp_1) * 100
        inflation = (deflator_1 / deflator_0 - 1) * 100


    # Check approximation: gN approx = gR + inflation
    approx_check = g_real + inflation if not (np.isnan(g_real) or np.isnan(inflation)) else np.nan

    # --- Plotting ---
    years = ['Year 0 (Base)', 'Year 1']
    nominal_values = [nom_gdp_0, nom_gdp_1]
    real_values = [real_gdp_0, real_gdp_1]

    fig, ax = plt.subplots(figsize=(8, 5))
    bar_width = 0.35
    index = np.arange(len(years))

    bars1 = ax.bar(index - bar_width/2, nominal_values, bar_width, label=f'Nominal GDP (Growth={g_nominal:.1f}%)' if not np.isnan(g_nominal) else 'Nominal GDP', color='#ff7f0e', alpha=0.8, edgecolor='black')
    bars2 = ax.bar(index + bar_width/2, real_values, bar_width, label=f'Real GDP (Growth={g_real:.1f}%)' if not np.isnan(g_real) else 'Real GDP', color='#1f77b4', alpha=0.8, edgecolor='black')

    ax.set_title("Nominal vs. Real GDP (Base-Year Method)")
    ax.set_ylabel("GDP ($)")
    ax.set_xticks(index)
    ax.set_xticklabels(years)
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    # Add bar labels
    ax.bar_label(bars1, fmt='$%.0f', padding=3, fontsize=9)
    ax.bar_label(bars2, fmt='$%.0f', padding=3, fontsize=9)
    # Set y-limit to start at 0
    ax.set_ylim(bottom=0)

    plt.tight_layout()
    plt.show()

    # --- Display Results ---
    # Helper to format NaN
    def format_val(val, fmt):
        return fmt.format(val) if not np.isnan(val) else "N/A"

    results_md = f"""
    ### 📊 GDP Calculation Summary (Base Year = Year 0):

    **Inputs:**
    | Item      | Yr 0 Qty | Yr 0 Price | Yr 1 Qty | Yr 1 Price |
    |-----------|----------|------------|----------|------------|
    | Good 1    | {q_good1_0}    | ${p_good1_0:.2f}   | {q_good1_1}    | ${p_good1_1:.2f}   |
    | Good 2    | {q_good2_0}    | ${p_good2_0:,.2f}  | {q_good2_1}    | ${p_good2_1:,.2f}  |

    **Results:**
    * **Nominal GDP Year 0:** ${nom_gdp_0:,.2f}$
    * **Nominal GDP Year 1:** ${nom_gdp_1:,.2f}$
    * **Real GDP Year 0 (Base=Y0):** ${real_gdp_0:,.2f}$
    * **Real GDP Year 1 (Base=Y0):** ${real_gdp_1:,.2f}$
    * **Nominal Growth Rate (gN):** {format_val(g_nominal, '{:.2f}%')}
    * **Real Growth Rate (gR):** **{format_val(g_real, '{:.2f}%')}**
    * **GDP Deflator Year 1 (Base=100):** {format_val(deflator_1, '{:.1f}')}
    * **Inflation Rate (π ≈ %ΔDeflator):** {format_val(inflation, '{:.2f}%')}
    * **Check (gR + π):** {format_val(approx_check, '{:.2f}%')} (Should approx. = gN)
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': '100px'} # Adjust width
layout = Layout(width='98%') # Adjust overall width slightly

interact(
    compare_nom_real_gdp_two_goods,
    p_good1_0=FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='P1 (Yr 0):', style=style, layout=layout),
    q_good1_0=IntSlider(value=100, min=0, max=500, step=10, description='Q1 (Yr 0):', style=style, layout=layout),
    p_good2_0=FloatSlider(value=500.0, min=1, max=2000, step=10, description='P2 (Yr 0):', style=style, layout=layout, readout_format=',.0f'),
    q_good2_0=IntSlider(value=5, min=0, max=50, step=1, description='Q2 (Yr 0):', style=style, layout=layout),
    p_good1_1=FloatSlider(value=3.0, min=0.1, max=10.0, step=0.1, description='P1 (Yr 1):', style=style, layout=layout),
    q_good1_1=IntSlider(value=120, min=0, max=500, step=10, description='Q1 (Yr 1):', style=style, layout=layout),
    p_good2_1=FloatSlider(value=550.0, min=1, max=2000, step=10, description='P2 (Yr 1):', style=style, layout=layout, readout_format=',.0f'),
    q_good2_1=IntSlider(value=6, min=0, max=50, step=1, description='Q2 (Yr 1):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=2.0, description='P1 (Yr 0):', layout=Layout(width='98%'), max=10.0, m…